.

Data_Engineering_TIL(20201101)

### [문제상황]

어떤 spark application이 이미 kill되어 있는 상태임에도 불구하고 알수없는 어떤 이유로 HDFS에 계속 뭔가 작업을 try해서 아래와 같은 hdfs log가 지속적으로 쌓이는 현상이 발생했다. 그래서 `/mnt/var/log/hadoop-hdfs` 가 급속도록 증가해서 `/mnt` 가 할당된 용량을 초과했고, 이로인해 jupyterhub 서비스가 다운되어 버렸다.

In [ ]:
2020-11-01 05:28:23,978 INFO org.apache.hadoop.hdfs.server.namenode.FSNamesystem (org.apache.hadoop.hdfs.server.namenode.LeaseManager$Monitor@abcd908): Recovering [Lease.  Holder: DFSClient_NONMAPREDUCE_-000000000_32, pending creates: 1], src=/var/log/spark/apps/application_000000000000_0047
2020-11-01 05:28:23,978 WARN org.apache.hadoop.hdfs.StateChange (org.apache.hadoop.hdfs.server.namenode.LeaseManager$Monitor@abbc908): DIR* NameSystem.internalReleaseLease: Failed to release lease for file /var/log/spark/apps/application_000000000000_0047. Committed blocks are waiting to be minimally replicated. Try again later.
2020-11-01 05:28:23,978 WARN org.apache.hadoop.hdfs.server.namenode.LeaseManager (org.apache.hadoop.hdfs.server.namenode.LeaseManager$Monitor@abcd908): Cannot release the path /var/log/spark/apps/application_000000000000_0047 in the lease [Lease.  Holder: DFSClient_NONMAPREDUCE_-000000000_32, pending creates: 1]. It will be retried.
org.apache.hadoop.hdfs.protocol.AlreadyBeingCreatedException: DIR* NameSystem.internalReleaseLease: Failed to release lease for file /var/log/spark/apps/application_1603793534410_0047. Committed blocks are waiting to be minimally replicated. Try again later.
        at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.internalReleaseLease(FSNamesystem.java:3482)
        at org.apache.hadoop.hdfs.server.namenode.LeaseManager.checkLeases(LeaseManager.java:586)
        at org.apache.hadoop.hdfs.server.namenode.LeaseManager$Monitor.run(LeaseManager.java:524)
        at java.lang.Thread.run(Thread.java:748)
2020-11-01 05:28:23,978 INFO org.apache.hadoop.hdfs.server.namenode.LeaseManager (org.apache.hadoop.hdfs.server.namenode.LeaseManager$Monitor@abcd908): [Lease.  Holder: DFSClient_NONMAPREDUCE_-000000000_32, pending creates: 1] has expired hard limit

### [해결방안]

#### hdfs와 jupyterhub 서비스를 restart하고, `/mnt` 볼륨을 기존에 약 50GB로 잡혀있는 용량을 256GB로 확장시킨다.

step 1) hdfs 재시작

마스터노드에서 아래와 같은 명령어 실행

`sudo systemctl restart hadoop-hdfs-namenode`

step 2) `/mnt` 볼륨을 256GB로 확장

먼저 아래와 같이 `/mnt` 볼륨의 용량과 시리얼 넘버를 확인한다.

`/mnt` 볼륨이 `nvme1n1`인 것을 확인할 수 있고, 기존의 용량은 51GB이고, 시리얼 넘버는 vol123456이다.

In [ ]:
[root@ip-10-0-5-185 hadoop-hdfs]# lsblk -o +SERIAL
NAME          MAJ:MIN RM  SIZE RO TYPE MOUNTPOINT SERIAL
nvme1n1       259:0    0   51G  0 disk            vol123456
├─nvme1n1p1   259:7  0    5G  0 part /emr
└─nvme1n1p2   259:8  0   46G  0 part /mnt
nvme2n1       259:1    0   64G  0 disk /mnt1      vol987654
nvme3n1       259:2    0   64G  0 disk /mnt2      vol456789
nvme4n1       259:3    0   64G  0 disk /mnt3      vol005518
nvme0n1       259:4    0  100G  0 disk            vol087150
├─nvme0n1p1   259:5    0   10G  0 part /
└─nvme0n1p128 259:6    0    1M  0 part

그런 다음에 AWS ebs의 volume 콘솔로 이동해서 `vol-123456`를 검색하고 해당 볼륨을 256GB로 리사이징해준다.

** 참고사항 : 기존의 볼륨을 늘리는 것은 가능한데 줄이는 것은 불가하다.

`vol-123456`이 256GB로 리사이징이 완료되면 다시 마스터 노드로 돌아와서 아래와 같은 명령어를 실행하여 볼륨을 마운트 시켜준다.

In [ ]:
[root@ip-10-0-5-185 hadoop-hdfs]# sudo growpart /dev/nvme1n1 2

[root@ip-10-0-5-185 hadoop-hdfs]# sudo xfs_growfs -d /mnt


# /mnt 가 정상적으로 확장이 되었는지 확인

[root@ip-10-0-5-185 hadoop-hdfs]# lsblk -o +SERIAL
NAME          MAJ:MIN RM  SIZE RO TYPE MOUNTPOINT SERIAL
nvme1n1       259:0    0  256G  0 disk            vol123456
├─nvme1n1p1   259:7    0    5G  0 part /emr
└─nvme1n1p2   259:8    0  251G  0 part /mnt
nvme2n1       259:1    0   64G  0 disk /mnt1      vol987654
nvme3n1       259:2    0   64G  0 disk /mnt2      vol456789
nvme4n1       259:3    0   64G  0 disk /mnt3      vol005518
nvme0n1       259:4    0  100G  0 disk            vol087150
├─nvme0n1p1   259:5    0   10G  0 part /
└─nvme0n1p128 259:6    0    1M  0 part

step 3) jupyterhub 서비스 재시작

마스터 노드에서 아래의 명령어를 실행

`sudo docker jupyterhub restart`